### 神经网络的梯度
·考虑如下有 d 层的神经网络

$$\mathbf{h}^t=f_t(\mathbf{h}^{t-1})\quad\mathrm{and}\quad y=\ell\circ f_d\circ...\circ f_1(\mathbf{x})$$
·计算损失$\ell\text{关于参数W}_t$的梯度
$$\frac{\partial\ell}{\partial\mathbf{W}^t}=\frac{\partial\ell}{\partial\mathbf{h}^d}\frac{\partial\mathbf{h}^d}{\partial\mathbf{h}^{d-1}}...\frac{\partial\mathbf{h}^{t+1}}{\partial\mathbf{h}^t}\frac{\partial\mathbf{h}^t}{\partial\mathbf{W}^t}$$

- 例子： MLP ·加入如下 MLP (为了简单省略了偏移)

$$
f_t(\mathbf{h}^{t-1})=\sigma(\mathbf{W}^t\mathbf{h}^{t-1})\ \ \  \sigma 是激活函数
$$

$$

\frac{\partial\mathbf{h}^{t}}{\partial\mathbf{h}^{t-1}}=\mathbf{diag}\left(\sigma^{\prime}(\mathbf{W}^{t}\mathbf{h}^{t-1})\right)(\dot{W}^{t})^{T}\quad\sigma^{\prime}\text{是}\sigma\text{的导数函数}
\\
\prod_{i=t}^{d-1}\frac{\partial\mathbf{h}^{i+1}}{\partial\mathbf{h}^i}=\prod_{i=t}^{d-1}\text{diag}\left(\sigma^{\prime}(\mathbf{W}^i\mathbf{h}^{i-1})\right)(W^i)^T
$$

·使用 ReLU 作为激活函数

$$\sigma( x) = \max ( 0, x) \quad $and$\quad \sigma^{\prime}( x) = \begin{cases} 1& \text{if}x> 0\\ 0& \text{otherwise}& \end{cases} $$
$$.\prod_{i=t}^{d-1}\frac{\partial\mathbf{h}^{i+1}}{\partial\mathbf{h}^{i}}=\prod_{i=t}^{d-1}\mathrm{diag}\left(\sigma(\mathbf{W}^{i}\mathbf{h}^{i-1})\right)(W^{i})^{T}\text{的一些元素会来自于}\prod_{i=t}^{d-1}(W^{i})^{T}$$
如果 d-t 很大，值将会很大


## 梯度爆炸的问题
值超出值域(infinity)
对于16位浮点数尤为严重(数值区间 6e-5-6e4
对学习率敏感
如果学习率太大-> 大参数值 -> 更大的梯度
如果学习率太小->训练无进展
我们可能需要在训练过程不断调整学习率

·目标： 让梯度值在合理的范围内

·目标:让梯度值在合理的范围内
   ·例如[le-6,1e3]
· 将乘法变加法
    ·ResNet,LSTM
·归一化
    梯度归一化，梯度裁剪
合理的权重初始和激活函数

让每层的方差是一个常数
将每层的输出和梯度都看做随机变量。
让它们的均值和方差都保持一致

$\begin{aligned}&\text{正向}\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \text{反向}\\&\mathbb{E}[h_i^t]=0\\&\text{Var}[h_i^t]=a\end{aligned}\quad\mathbb{E}\left[\frac{\partial\ell}{\partial h_i^t}\right]=0\quad\text{Var}\left[\frac{\partial\ell}{\partial h_i^t}\right]=b\quad\forall i,t$

$\text{а 和 b 都是常数}$


例子:MLP
假设
$w_{i,j}^t$ 是 i.i.d, 那么E[w$_{i, j}^t] = 0, $ Var$[ w_{i, j}^t] = \gamma_t$

· $h_i^{t-1}$ 独立于 $w_{i,j}^t$
$\boldsymbol{\cdot}\text{ 假设没有激活函数 }\mathbf{h}^{\prime}=\mathbf{W}^t\mathbf{h}^{t-1}\text{,这里 }\mathbf{W}^t\in\mathbb{R}^{n,\times n_{t-1}}$

$\mathbb{E}[h_i^t]=\mathbb{E}\left[\sum_jw_{i,j}^th_j^{t-1}\right]=\sum_j\mathbb{E}[w_{i,j}^t]\mathbb{E}[h_j^{t-1}]=0$


$\text{正向方差}$

$$
\begin{aligned}
\mathbf{Var}[h_{i}^{t}]& =\mathbb{E}[(h_i^t)^2]-\mathbb{E}[h_i^t]^2=\mathbb{E}\left[\left(\sum_jw_{i,j}^th_j^{t-1}\right)^2\right]  \\
&=\mathbb{E}\left[\sum_j\left(w_{i,j}^t\right)^2\left(h_j^{t-1}\right)^2+\sum_{j\neq k}w_{i,j}^tw_{i,k}^th_j^{t-1}h_k^{t-1}\right] \\
&=\sum_j\mathbb{E}\left[\left(w_{i,j}^t\right)^2\right]\mathbb{E}\left[\left(h_j^{t-1}\right)^2\right] \\
&=\sum_j\mathrm{Var}[w_{i,j}^t]\mathrm{Var}[h_j^{t-1}]=n_{t-1}\gamma_t\mathrm{Var}[h_j^{t-1}]
\end{aligned}
\\
\\
n_{t-1}\gamma_t=1
$$

反向均值和方差
跟正向情况类似
$$\begin{aligned}
&\frac{\partial\ell}{\partial\mathbf{h}^{t-1}}=\frac{\partial\ell}{\partial\mathbf{h}^{t}}\mathbf{W}^{t}→\quad\left(\frac{\partial\ell}{\partial\mathbf{h}^{t-1}}\right)^{T}=(W^{t})^{T}\bigg(\frac{\partial\ell}{\partial\mathbf{h}^{t}}\bigg)^{T} \\
&\mathbb{E}\left[\frac{\partial\ell}{\partial h_i^{t-1}}\right]=0 \\
&\mathrm{Var}\left[\frac{\partial\ell}{\partial h_{i}^{t-1}}\right]=n_{t}\gamma_{t}\mathrm{Var}\left[\frac{\partial\ell}{\partial h_{j}^{t}}\right]\quad →n_{t}\gamma_{t}=1
\end{aligned}$$

### Xavier初始
难以需要满足$n_{t-1}\gamma_t=1$和$n_t\gamma_t=1$
·Xavier 使得 $\gamma_t(n_{t-1}+n_t)/2=1\quad\to\gamma_t=2/(n_{t-1}+n_t)$
 · 正态分布 $\mathcal{N}\left(0,\sqrt{2l(n_{t-1}+n_t)}\right)$
 . 均匀分布 $\mathcal{U}\left(-\sqrt{6/(n_{t-1}+n_t)},\sqrt{6/(n_{t-1}+n_t)}\right)$
 ·分布 $\mathcal{U}_{[-a,a]}$ 和方差是 $a^2/3$

·适配权重形状变换，特别是$n_t$

### 假设线性的激活函数
$$\begin{aligned}
\bullet\text{ 假设 }\sigma(x)& =\alpha x+\beta   \\
&\mathbf{h}^{\prime}=\mathbf{W}^{t}\mathbf{h}^{t-1}\quad\mathbf{and}\quad\mathbf{h}^{t}=\sigma(\mathbf{h}^{\prime}) \\
\mathbb{E}[h_i^t]&   =\mathbb{E}\left[\alpha h_{i}^{\prime}+\beta\right]=\beta\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\beta=0  \\
\operatorname{Var}[h_{i}^{t}]&   \\
& =\mathbb{E}[(\alpha h_{i}^{\prime}+\beta)^{2}]-\beta^{2}\quad\quad\quad\quad\quad\quad\quad\quad\quad\quad\alpha=1    \\
&=\mathbb{E}[\alpha^2(h_i^{\prime})^2+2\alpha\beta h_i^{\prime}+\beta^2]-\beta^2 \\
&=\alpha^2\text{Var}[h_i^{\prime}]
\end{aligned}$$

### 反向
$$\begin{aligned}\bullet\text{ 假设 }\sigma(x)&=\alpha x+\beta\\\frac{\partial\ell}{\partial\mathbf{h}^{\prime}}&=\frac{\partial\ell}{\partial\mathbf{h}^{\prime}}(W^t)^T\quad\mathrm{and}\quad\frac{\partial\ell}{\partial\mathbf{h}^{t-1}}=\alpha\frac{\partial\ell}{\partial\mathbf{h}^{\prime}}\end{aligned}\\
\begin{aligned}&\mathbb{E}\left[\frac{\partial\ell}{\partial h_i^{t-1}}\right]=0&&\color{red}{\boxed{\Rightarrow}}&\beta=0\\[2ex]&\mathrm{Var}\left[\frac{\partial\ell}{\partial h_i^{t-1}}\right]=\alpha^2\mathrm{Var}\left[\frac{\partial\ell}{\partial h_j^{\prime}}\right]&&\color{red}{\boxed{\Rightarrow}}&\alpha=1\end{aligned}
$$
### 检查激活函数
使用泰勒展开
$$
\mathrm{sigmoid}(x)=\frac12+\frac x4-\frac{x^3}{48}+O(x^5)\\
\tanh(x)=0+x-\frac{x^3}3+O(x^5)\\
\mathrm{relu}(x)=0+x\quad\mathrm{for}x\geq0\\
$$

·调整 sigmoid:
$$4\times\mathrm{sigmoid}(x)-2$$